In [31]:
import numpy as np
class FeatureExtraction:
    def __init__(self, orginal_signal, windowlength, overlap):
        self.orginal_signal = orginal_signal
        self.windowlength = windowlength
        self.overlap = overlap
        self.indices = self.ini()
    def ini(self):
        delta = self.windowlength - self.overlap
        indices = np.arange(0, len(self.orginal_signal), delta)
        
        # Zeropad signal
        if len(self.orginal_signal) - indices[-1] + 1 < self.windowlength:
            self.orginal_signal = np.pad(self.orginal_signal, (0, indices[-1]+self.windowlength-1-len(self.orginal_signal)), 'constant')
        else:
            indices = indices[indices+self.windowlength-1 <= len(self.orginal_signal)]
        return(indices)
    def all_features(self):
        features = []
        features.append(self.RMS_func())
        features.append(self.SSI_func())
        features.append(self.STD_func())
        features.append(self.WL_func())
        features.append(self.MAV_func())
        features.append(self.MAV1_func())
        features.append(self.SSC_func())
        features.append(self.DASDV_func())
        features.append(self.LD_func())
        features.append(self.AAC_func())
        features.append(self.VAR_func())
        features.append(self.EMAV_func())
        features.append(self.EWL_func())
        features.append(self.V_OD_func())
        return(np.asarray(features))

    def RMS_func(self):
        # Root Mean Square
        RMS = np.zeros(len(self.indices))
        signal = self.orginal_signal**2
        for i, idx in enumerate(self.indices):
            RMS[i] = np.sqrt(np.mean(signal[idx:idx+self.windowlength]))
        return RMS
    def SSI_func(self):
        index = 0
        SSI = [0] * len(self.indices)
        signal = self.orginal_signal

        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            
            for i in range(N):
                Y += (sig[i])**2
            
            SSI[index-1] = Y
        return(SSI)
    def STD_func(self):
        index = 0
        STD = [0] * len(self.indices)
        signal = self.orginal_signal

        for j in self.indices:
            sig = signal[j:j + self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            meansig = sum(sig) / N
            
            for i in range(N):
                Y += (sig[i] - meansig)**2
            
            STD[index-1] = np.sqrt(Y / (N-1))
        return(STD)
    def WL_func(self):
        # Waveform Length
        index = 0
        WL = np.zeros(len(self.indices))
        signal = self.orginal_signal
        for j in self.indices:
            index += 1
            N = len(signal[j:j+self.windowlength])
            sig = signal[j:j+self.windowlength]
            for i in range(1, N):
                WL[index-1] += abs(sig[i]-sig[i-1])

        return WL
    def MAV_func(self):
        index = 0
        MAV = [0] * len(self.indices)
        signal = np.copy(self.orginal_signal)

        for j in self.indices:
            index += 1
            MAV[index-1] = np.mean(np.abs(signal[j:j+self.windowlength]))
        return(MAV)
    def MAV1_func(self):
        # Modified Mean Absolute Value Type 1
        index = 0
        MAV1 = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            for i in range(N):
                if i+1 >= 0.25*N and i+1 <= 0.75*N:
                    w = 1 
                else:
                    w = 0.5
                Y += (w * np.abs(sig[i]))
            MAV1[index-1] = (1/N) * Y

        return MAV1
    def SSC_func(self, thres=0.3):
        # Slope Sign Change
        index = 0
        SSC = np.zeros(len(self.indices))
        signal = self.orginal_signal
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            SSC[index-1] = 0
            for i in range(1,N-1):
                if ((sig[i] > sig[i-1] and sig[i] > sig[i+1]) or
                    (sig[i] < sig[i-1] and sig[i] < sig[i+1])) and \
                    ((np.abs(sig[i]-sig[i+1]) >= thres) or
                    (np.abs(sig[i]-sig[i-1]) >= thres)):
                    SSC[index-1] += 1 
        return SSC
    def DASDV_func(self):
        # Difference in Absolute Standard Deviation Value
        index = 0
        DASDV = np.zeros(len(self.indices))
        signal = self.orginal_signal
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            for i in range(N-1):
                Y += (sig[i+1] - sig[i])**2
            DASDV[index-1] = np.sqrt(Y/(N-1))
        return DASDV
    def LD_func(self):
        index = 0
        LD = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)

        for j in self.indices:
            sig = signal[j:j + self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            
            for k in range(N):
                Y += np.log(np.abs(sig[k]))
            
            LD[index-1] = np.exp(Y/N)
        return(LD)
    def AAC_func(self):#
        index = 0
        AAC = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)

        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            Y = 0
            
            for i in range(N-1):
                Y += abs(sig[i+1] - sig[i])
                
            AAC[index-1] = Y/N
        return(AAC)
    def VAR_func(self):
        # Variance of EMG
        index = 0
        VAR = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)
        
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            
            VAR[index-1] = (1/(N-1))*np.sum(sig**2)
        return(VAR)
    def EMAV_func(self):
        # Enhanced Mean absolute value
        index = 0
        EMAV = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)

        for j in self.indices:
            sig = signal[j:j + self.windowlength]
            index += 1
            L = len(sig)
            Y = np.zeros(L)
            
            for i in range(L):
                if i >= 0.2*L and i <= 0.8*L:
                    p = 0.75
                else:
                    p = 0.5
                
                Y[i] = abs(sig[i])**p
            
            EMAV[index-1] = np.mean(Y)
        return(EMAV)
    def EWL_func(self):
        # Enhanced Wavelength
        index = 0
        EWL = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            L = len(sig)
            p = np.where(np.logical_and(np.greater_equal(np.arange(2, L+1), 0.2*L), np.less_equal(np.arange(2, L+1), 0.8*L)), 0.75, 0.5)
            EWL[index-1] = np.sum(np.abs(np.diff(sig))**p)
        return(EWL)
    def V_OD_func(self):
        # V-Order
        index = 0
        V_OD = np.zeros(len(self.indices))
        signal = np.copy(self.orginal_signal)
        for j in self.indices:
            sig = signal[j:j+self.windowlength]
            index += 1
            N = len(sig)
            Y = np.sum(sig**2)
            V_OD[index-1] = np.power(Y/N, 1/2)
        
        return V_OD


In [32]:
f = FeatureExtraction(b, 10, 5).all_features()

C:\Users\kiano\AppData\Local\Temp/ipykernel_12928/1036006792.py:160: RuntimeWarning: divide by zero encountered in log
  Y += np.log(np.abs(sig[k]))


In [33]:
f.shape

(14, 200)

In [20]:
a = np.linspace(0,10,1000)
b = np.sin(a)
ss2 = FeatureExtraction(b, 10, 5).RMS_func()
ss3 = FeatureExtraction(b, 10, 5).SSI_func()
ss4 = FeatureExtraction(b, 10, 5).STD_func()
ss5 = FeatureExtraction(b, 10, 5).WL_func()
ss6 = FeatureExtraction(b, 10, 5).MAV_func()
ss7 = FeatureExtraction(b, 10, 5).MAV1_func()
ss8 = FeatureExtraction(b, 10, 5).SSC_func()
ss9 = FeatureExtraction(b, 10, 5).DASDV_func()
ss10 = FeatureExtraction(b, 10, 5).LD_func()
ss11 = FeatureExtraction(b, 10, 5).AAC_func()
ss12 = FeatureExtraction(b, 10, 5).VAR_func()
ss13 = FeatureExtraction(b, 10, 5).EMAV_func()
ss14 = FeatureExtraction(b, 10, 5).EWL_func()
ss15 = FeatureExtraction(b, 10, 5).V_OD_func()



C:\Users\kiano\AppData\Local\Temp/ipykernel_12928/4084081370.py:142: RuntimeWarning: divide by zero encountered in log
  Y += np.log(np.abs(sig[k]))
